<a href="https://colab.research.google.com/github/med925/CollectorRT/blob/master/Copy_of_the_new_model_cnn_csi%2Brssi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import math
import numpy as np
import pandas as pd
import requests
import os
import random
import tensorflow as tf
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import (LSTM, Dense, Flatten, Conv1D, Conv2D, MaxPooling2D, Dropout,
                                     BatchNormalization, GRU, GaussianNoise, DepthwiseConv2D, AveragePooling2D)
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from torch.utils.data import DataLoader, Dataset, TensorDataset
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dropout, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback, EarlyStopping

# Set a seed value
seed_value= 123

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

# For later versions: use `tf.compat.v1.set_random_seed(seed_value)`

In [2]:
# arnaud
url_arnaud_no_activity_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=34B47219ED40&start_date=2024-01-24%2014:00:00&end_date=2024-01-24%2015:28:32&message=NO_ACTIVITY"
url_arnaud_walk_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=34B47219ED40&start_date=2024-01-24%2014:00:00&end_date=2024-01-24%2015:28:32&message=WALK"
url_arnaud_run_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=34B47219ED40&start_date=2024-01-24%2014:00:00&end_date=2024-01-24%2015:28:32&message=RUN"
url_arnaud_no_activity_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=34B47219ED40&start_date=2024-01-24%2015:29:00&end_date=2024-01-24%2015:42:36&message=NO_ACTIVITY"
url_arnaud_walk_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=34B47219ED40&start_date=2024-01-24%2015:29:00&end_date=2024-01-24%2015:42:36&message=WALK"
url_arnaud_run_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=34B47219ED40&start_date=2024-01-24%2015:29:00&end_date=2024-01-24%2015:42:36&message=RUN"
url_arnaud_no_activity_ch_3 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=34B47219ED40&start_date=2024-01-24%2015:43:00&end_date=2024-01-24%2016:00:36&message=NO_ACTIVITY"
url_arnaud_walk_ch_3 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=34B47219ED40&start_date=2024-01-24%2015:43:00&end_date=2024-01-24%2016:00:36&message=WALK"
url_arnaud_run_ch_3 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=34B47219ED40&start_date=2024-01-24%2015:43:00&end_date=2024-01-24%2016:00:36&message=RUN"

#siwar
url_siwar_no_activity_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=7CDFA1A43F54&start_date=2024-01-23%2020:13:00&end_date=2024-01-23%2020:51:17&message=NO_ACTIVITY"
url_siwar_walk_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=7CDFA1A43F54&start_date=2024-01-23%2020:13:00&end_date=2024-01-23%2020:51:17&message=WALK"
url_siwar_run_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=7CDFA1A43F54&start_date=2024-01-23%2020:13:00&end_date=2024-01-23%2020:51:17&message=RUN"

url_siwar_no_activity_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=7CDFA1A43F54&start_date=2024-01-23%2021:02:00&end_date=2024-01-23%2021:26:00&message=NO_ACTIVITY"
url_siwar_walk_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=7CDFA1A43F54&start_date=2024-01-23%2021:02:00&end_date=2024-01-23%2021:26:00&message=WALK"
url_siwar_run_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=7CDFA1A43F54&start_date=2024-01-23%2021:02:00&end_date=2024-01-23%2021:26:00&message=RUN"

#ahmed
url_ahmed_no_activity_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA050&start_date=2024-01-23%2021:26:00&end_date=2024-01-23%2022:30:28&message=NO_ACTIVITY"
url_ahmed_walk_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA050&start_date=2024-01-23%2021:26:00&end_date=2024-01-23%2022:30:28&message=WALK"
url_ahmed_run_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA050&start_date=2024-01-23%2021:26:00&end_date=2024-01-23%2022:30:28&message=RUN"

url_ahmed_no_activity_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA050&start_date=2024-01-23%2022:31:00&end_date=2024-01-23%2023:01:22&message=NO_ACTIVITY"
url_ahmed_walk_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA050&start_date=2024-01-23%2022:31:00&end_date=2024-01-23%2023:01:22&message=WALK"
url_ahmed_run_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA050&start_date=2024-01-23%2022:31:00&end_date=2024-01-23%2023:01:22&message=RUN"

url_ahmed_no_activity_ch_3 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA050&start_date=2024-01-23%2023:02:00&end_date=2024-01-24%2000:13:14&message=NO_ACTIVITY"
url_ahmed_walk_ch_3 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA050&start_date=2024-01-23%2023:02:00&end_date=2024-01-24%2000:13:14&message=WALK"
url_ahmed_run_ch_3 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA050&start_date=2024-01-23%2023:02:00&end_date=2024-01-24%2000:13:14&message=RUN"

#olivier
url_olivierB_no_activity_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8A2BF8&start_date=2024-01-25%2014:34:00&end_date=2024-01-25%2015:00:00&message=NO_ACTIVITY"
url_olivierB_walk_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8A2BF8&start_date=2024-01-25%2014:34:00&end_date=2024-01-25%2015:00:00&message=WALK"
url_olivierB_run_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8A2BF8&start_date=2024-01-25%2014:34:00&end_date=2024-01-25%2015:00:00&message=RUN"

#Pet
url_adeline_pet_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA88A554&start_date=2024-02-02%2007:56:26&end_date=2024-02-02%2011:06:53&message=PET"
url_adeline_pet2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA88A554&start_date=2024-02-02%2011:06:54&end_date=2024-02-02%2020:06:53&message=PET"
url_ahmed_pet_1_face_to_face = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA050&start_date=2024-02-01%2007:26:00&end_date=2024-02-01%2012:05:40&message=PET"
url_ahmed_pet_1_diag = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA050&start_date=2024-02-01%2012:06:00&end_date=2024-02-01%2012:17:11&message=PET"
url_ahmed_pet_2_diag = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA050&start_date=2024-02-01%2012:18:00&end_date=2024-02-01%2012:34:39&message=PET"
url_ahmed_pet_3_diag = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA050&start_date=2024-02-01%2012:34:40&end_date=2024-02-01%2012:43:40&message=PET"
url_anica_pet = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8C1384&start_date=2024-02-02%2007:06:54&end_date=2024-02-02%2013:28:59&message=PET"
url_benjamin_pet_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8E1D28&start_date=2024-02-07%2007:06:54&end_date=2024-02-07%2013:28:59&message=PET"
url_benjamin_pet_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8E1D28&start_date=2024-02-07%2013:29:00&end_date=2024-02-07%2020:28:59&message=PET"


In [3]:
#cederic
url_cedric_no_activity_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA88A11C&start_date=2024-02-27%2007:13:00&end_date=2024-02-27%2020:51:17&message=NO_ACTIVITY"
url_cedric_walk_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA88A11C&start_date=2024-02-27%2007:13:00&end_date=2024-02-27%2020:51:17&message=WALK"
url_cedric_run_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA88A11C&start_date=2024-02-27%2007:13:00&end_date=2024-02-27%2020:51:17&message=RUN"
url_cedric_pet_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA88A11C&start_date=2024-02-27%2007:13:00&end_date=2024-02-27%2020:51:17&message=PET"
url_cedric_pet_human_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA88A11C&start_date=2024-02-27%2007:13:00&end_date=2024-02-27%2020:51:17&message=HP"

#julien
url_julien_walk_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8C0F30&start_date=2024-02-23%2013:50:00&end_date=2024-02-23%2020:44:17&message=WALK"
url_julien_run_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8C0F30&start_date=2024-02-23%2013:50:00&end_date=2024-02-23%2020:44:17&message=RUN"
url_julien_pet_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8C0F30&start_date=2024-02-23%2013:50:00&end_date=2024-02-23%2020:44:17&message=PET"
url_julien_pet_human_ch_2 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8C0F30&start_date=2024-02-23%2013:50:00&end_date=2024-02-23%2020:44:17&message=HP"

url_julien_no_activity_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8C0F30&start_date=2024-02-23%2007:41:00&end_date=2024-02-23%2013:44:17&message=NO_ACTIVITY"
url_julien_walk_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8C0F30&start_date=2024-02-23%2007:41:00&end_date=2024-02-23%2013:44:17&message=WALK"
url_julien_run_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8C0F30&start_date=2024-02-23%2007:41:00&end_date=2024-02-23%2013:44:17&message=RUN"
url_julien_pet_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8C0F30&start_date=2024-02-23%2007:41:00&end_date=2024-02-23%2013:44:17&message=PET"
url_julien_pet_human_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8C0F30&start_date=2024-02-23%2007:41:00&end_date=2024-02-23%2013:44:17&message=HP"

#johanna
url_Johanna_no_activity_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EAA00&start_date=2024-02-23%2007:29:02&end_date=2024-02-23%2014:50:00&message=NO_ACTIVITY"
url_Johanna_walk_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EAA00&start_date=2024-02-23%2007:29:02&end_date=2024-02-23%2014:50:00&message=WALK"
url_Johanna_run_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EAA00&start_date=2024-02-23%2007:29:02&end_date=2024-02-23%2014:50:00&message=RUN"
url_Johanna_pet_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EAA00&start_date=2024-02-23%2007:29:02&end_date=2024-02-23%2014:50:00&message=PET"
url_Johanna_pet_human_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EAA00&start_date=2024-02-23%2007:29:02&end_date=2024-02-23%2014:50:00&message=HP"

#ambre
url_ambre_no_activity_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EBD84&start_date=2024-02-28%2007:13:00&end_date=2024-02-28%2020:51:17&message=NO_ACTIVITY"
url_ambre_walk_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EBD84&start_date=2024-02-28%2007:13:00&end_date=2024-02-28%2020:51:17&message=WALK"
url_ambre_run_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EBD84&start_date=2024-02-28%2007:13:00&end_date=2024-02-28%2020:51:17&message=RUN"
url_ambre_pet_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EBD84&start_date=2024-02-28%2007:13:00&end_date=2024-02-28%2020:51:17&message=PET"
url_ambre_pet_human_ch_1 = "https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EBD84&start_date=2024-02-28%2007:13:00&end_date=2024-02-28%2020:51:17&message=HP"

#emeric
url_st_emeric_No_activity ="https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA88C&start_date=2024-02-28%2013:50:00&end_date=2024-02-28%2017:22:17&message=NO_ACTIVITY"
url_st_emeric_walk ="https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA88C&start_date=2024-02-28%2013:50:00&end_date=2024-02-28%2017:25:17&message=WALK"
url_st_emeric_run ="https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA88C&start_date=2024-02-28%2013:50:00&end_date=2024-02-28%2017:28:17&message=RUN"
url_st_emeric_chien1_t1="https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA88C&start_date=2024-02-28%2013:50:00&end_date=2024-02-28%2017:32:59&message=PET"
url_st_emeric_chien1_t2="https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA88C&start_date=2024-02-28%2017:40:00&end_date=2024-02-28%2017:50:00&message=PET"
url_st_emeric_chien2_t1="https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA88C&start_date=2024-02-28%2017:33:00&end_date=2024-02-28%2017:39:56&message=PET"
url_st_emeric_pet_human_chien1="https://traci.tn/cu/ptl.php?limit=100000000000&topic=404CCA8EA88C&start_date=2024-02-28%2017:48:00&end_date=2024-02-28%2018:00:00&message=HP"

In [4]:
#Pet
#ahmed
response_ahmed_pet_1 = requests.get(url_ahmed_pet_1_face_to_face)
response_ahmed_pet_2 = requests.get(url_ahmed_pet_1_diag)
response_ahmed_pet_3 = requests.get(url_ahmed_pet_2_diag)
response_ahmed_pet_4 = requests.get(url_ahmed_pet_3_diag)

string_ahmed_pet_ch_1 = response_ahmed_pet_1.text
string_ahmed_pet_ch_2 = response_ahmed_pet_2.text
string_ahmed_pet_ch_3 = response_ahmed_pet_3.text
string_ahmed_pet_ch_4 = response_ahmed_pet_4.text
#anica
response_anica_pet_1 = requests.get(url_anica_pet)
string_anica_pet_ch_1 = response_anica_pet_1.text
#benjamin
response_benjamin_pet_1 = requests.get(url_benjamin_pet_ch_1)
response_benjamin_pet_2 = requests.get(url_benjamin_pet_ch_2)
string_benjamin_pet_ch_1 = response_benjamin_pet_1.text
string_benjamin_pet_ch_2 = response_benjamin_pet_2.text
#cederic
response_cederic_pet_1 = requests.get(url_cedric_pet_ch_1)
string_cederic_pet_ch_1 = response_cederic_pet_1.text
#julien
response_julien_pet_2 = requests.get(url_julien_pet_ch_2)
response_julien_pet_1 = requests.get(url_julien_pet_ch_1)
string_julien_pet_ch_1 = response_julien_pet_1.text
string_julien_pet_ch_2 = response_julien_pet_2.text
#johanna
response_johanna_pet_1 = requests.get(url_Johanna_pet_ch_1)
string_johanna_pet_ch_1 = response_johanna_pet_1.text
#ambre
response_ambre_pet_1 = requests.get(url_ambre_pet_ch_1)
string_ambre_pet_ch_1 = response_ambre_pet_1.text
#emeric
response_emeric_pet_1 = requests.get(url_st_emeric_chien1_t1)
response_emeric_pet_2 = requests.get(url_st_emeric_chien1_t2)
response_emeric_pet_3 = requests.get(url_st_emeric_chien2_t1)

string_emeric_pet_ch_1 = response_emeric_pet_1.text
string_emeric_pet_ch_2 = response_emeric_pet_2.text
string_emeric_pet_ch_3 = response_emeric_pet_3.text

In [5]:
#arnaud
response_arnaud_no_activity_ch_1 = requests.get(url_arnaud_no_activity_ch_1)
response_arnaud_walk_ch_1 = requests.get(url_arnaud_walk_ch_1)
response_arnaud_run_ch_1 = requests.get(url_arnaud_run_ch_1)

response_arnaud_no_activity_ch_2 = requests.get(url_arnaud_no_activity_ch_2)
response_arnaud_walk_ch_2 = requests.get(url_arnaud_walk_ch_2)
response_arnaud_run_ch_2 = requests.get(url_arnaud_run_ch_2)

response_arnaud_no_activity_ch_3 = requests.get(url_arnaud_no_activity_ch_3)
response_arnaud_walk_ch_3 = requests.get(url_arnaud_walk_ch_3)
response_arnaud_run_ch_3 = requests.get(url_arnaud_run_ch_3)

string_arnaud_no_activity_ch_1 = response_arnaud_no_activity_ch_1.text
string_arnaud_walk_ch_1 = response_arnaud_walk_ch_1.text
string_arnaud_run_ch_1 = response_arnaud_run_ch_1.text

string_arnaud_no_activity_ch_2 = response_arnaud_no_activity_ch_2.text
string_arnaud_walk_ch_2 = response_arnaud_walk_ch_2.text
string_arnaud_run_ch_2 = response_arnaud_run_ch_2.text

string_arnaud_no_activity_ch_3 = response_arnaud_no_activity_ch_3.text
string_arnaud_walk_ch_3 = response_arnaud_walk_ch_3.text
string_arnaud_run_ch_3= response_arnaud_run_ch_3.text

#ahmed
response_ahmed_no_activity_ch_1 = requests.get(url_ahmed_no_activity_ch_1)
response_ahmed_walk_ch_1 = requests.get(url_ahmed_walk_ch_1)
response_ahmed_run_ch_1 = requests.get(url_ahmed_run_ch_1)

response_ahmed_no_activity_ch_2 = requests.get(url_ahmed_no_activity_ch_2)
response_ahmed_walk_ch_2 = requests.get(url_ahmed_walk_ch_2)
response_ahmed_run_ch_2 = requests.get(url_ahmed_run_ch_2)

response_ahmed_no_activity_ch_3 = requests.get(url_ahmed_no_activity_ch_3)
response_ahmed_walk_ch_3 = requests.get(url_ahmed_walk_ch_3)
response_ahmed_run_ch_3 = requests.get(url_ahmed_run_ch_3)

string_ahmed_no_activity_ch_1 = response_ahmed_no_activity_ch_1.text
string_ahmed_walk_ch_1 = response_ahmed_walk_ch_1.text
string_ahmed_run_ch_1 = response_ahmed_run_ch_1.text

string_ahmed_no_activity_ch_2 = response_ahmed_no_activity_ch_2.text
string_ahmed_walk_ch_2 = response_ahmed_walk_ch_2.text
string_ahmed_run_ch_2 = response_ahmed_run_ch_2.text

string_ahmed_no_activity_ch_3 = response_ahmed_no_activity_ch_3.text
string_ahmed_walk_ch_3 = response_ahmed_walk_ch_3.text
string_ahmed_run_ch_3 = response_ahmed_run_ch_3.text

#siwar
response_siwar_no_activity_ch_1 = requests.get(url_siwar_no_activity_ch_1)
response_siwar_walk_ch_1 = requests.get(url_siwar_walk_ch_1)
response_siwar_run_ch_1 = requests.get(url_siwar_run_ch_1)

response_siwar_no_activity_ch_2 = requests.get(url_siwar_no_activity_ch_2)
response_siwar_walk_ch_2 = requests.get(url_siwar_walk_ch_2)
response_siwar_run_ch_2 = requests.get(url_siwar_run_ch_2)

string_siwar_no_activity_ch_1 = response_siwar_no_activity_ch_1.text
string_siwar_walk_ch_1 = response_siwar_walk_ch_1.text
string_siwar_run_ch_1 = response_siwar_run_ch_1.text

string_siwar_no_activity_ch_2 = response_siwar_no_activity_ch_2.text
string_siwar_walk_ch_2 = response_siwar_walk_ch_2.text
string_siwar_run_ch_2 = response_siwar_run_ch_2.text

#olivier
response_olivierB_no_activity_ch_2 = requests.get(url_olivierB_no_activity_ch_2)
response_olivierB_walk_ch_2 = requests.get(url_olivierB_walk_ch_2)
response_olivierB_run_ch_2 = requests.get(url_olivierB_run_ch_2)

string_olivierB_no_activity_ch_2 = response_olivierB_no_activity_ch_2.text
string_olivierB_walk_ch_2 = response_olivierB_walk_ch_2.text
string_olivierB_run_ch_2 = response_olivierB_run_ch_2.text




In [6]:
#human
#cederic
response_cederic_no_activity_ch_1 = requests.get(url_cedric_no_activity_ch_1)
response_cederic_walk_ch_1 = requests.get(url_cedric_walk_ch_1)
response_cederic_run_ch_1 = requests.get(url_cedric_run_ch_1)

string_cederic_no_activity_ch_1 = response_cederic_no_activity_ch_1.text
string_cederic_walk_ch_1 = response_cederic_walk_ch_1.text
string_cederic_run_ch_1 = response_cederic_run_ch_1.text

#julien
response_julien_no_activity_ch_1 = requests.get(url_julien_no_activity_ch_1)
response_julien_walk_ch_1 = requests.get(url_julien_walk_ch_1)
response_julien_run_ch_1 = requests.get(url_julien_run_ch_1)

string_julien_no_activity_ch_1 = response_julien_no_activity_ch_1.text
string_julien_walk_ch_1 = response_julien_walk_ch_1.text
string_julien_run_ch_1 = response_julien_run_ch_1.text

response_julien_walk_ch_2 = requests.get(url_julien_walk_ch_2)
response_julien_run_ch_2 = requests.get(url_julien_run_ch_2)

string_julien_walk_ch_2 = response_julien_walk_ch_2.text
string_julien_run_ch_2 = response_julien_run_ch_2.text

#johanna
response_johanna_no_activity_ch_1 = requests.get(url_Johanna_no_activity_ch_1)
response_johanna_walk_ch_1 = requests.get(url_Johanna_walk_ch_1)
response_johanna_run_ch_1 = requests.get(url_Johanna_run_ch_1)

string_johanna_no_activity_ch_1 = response_johanna_no_activity_ch_1.text
string_johanna_walk_ch_1 = response_johanna_walk_ch_1.text
string_johanna_run_ch_1 = response_johanna_run_ch_1.text

#ambre
response_ambre_no_activity_ch_1 = requests.get(url_ambre_no_activity_ch_1)
response_ambre_walk_ch_1 = requests.get(url_ambre_walk_ch_1)
response_ambre_run_ch_1 = requests.get(url_ambre_run_ch_1)

string_ambre_no_activity_ch_1 = response_ambre_no_activity_ch_1.text
string_ambre_walk_ch_1 = response_ambre_walk_ch_1.text
string_ambre_run_ch_1 = response_ambre_run_ch_1.text

#emeric
response_emeric_no_activity_ch_1 = requests.get(url_st_emeric_No_activity)
response_emeric_walk_ch_1 = requests.get(url_st_emeric_walk)
response_emeric_run_ch_1 = requests.get(url_st_emeric_run)

string_emeric_no_activity_ch_1 = response_emeric_no_activity_ch_1.text
string_emeric_walk_ch_1 = response_emeric_walk_ch_1.text
string_emeric_run_ch_1 = response_emeric_run_ch_1.text

In [7]:
# arrang data with timing considérations
def createDict(string, attributeJson, feature ='csi'):
    my_set =set()
    dict_timestamp ={}
    json_data = json.loads(string)
    for i in range(len(json_data)):
        message = json_data[i]["message"]
        message_json = json.loads(message)
        my_set.add(int(message_json["timestamp"]))
    for element in my_set:
        dict_timestamp[element] = []
    for i in range(len(json_data)):
        message = json_data[i]["message"]
        message_json = json.loads(message)
        dict_timestamp[int(message_json["timestamp"])].append(message_json)
    l= (list(dict_timestamp.keys()))
    (l.sort())
    dict_micro ={}
    dict_time_stamp ={}
    for element in l:
        for x in dict_timestamp[element]:
            list_csi = list(map(int, x['csi'].split(',')))
            if(len(list_csi)!=112 or x["rssi"]==''):
                continue
            #dict_micro[int(x["Micro"])] = list(map(int, x['csi'].split(',')))
            dict_micro[int(x[attributeJson])] = list(map(int, x[feature].split(',')))
        l_sort = list(dict_micro.keys())
        l_sort.sort()
        dict_micro_temp ={}
        for n in l_sort:
            dict_micro_temp[n] = dict_micro[n]
        dict_time_stamp[element] = dict_micro_temp
        dict_micro ={}
    return dict_time_stamp



In [8]:
#fix csi order
def fix_array(aerray):
    aerray = np.array(aerray)
    tempz=[]
    for i,aeerray in enumerate(aerray):
        arr_temp1 = aeerray[:,:26]
        arr_temp2 = aeerray[:,26:]
        arr = np.concatenate((arr_temp2,arr_temp1),axis=1)
        tempz.append(arr)
    return tempz

In [9]:
def rearrange_data(data, sample_size=56):
    # Calculate number of groups
    num_groups = len(data) // sample_size
    # Create the ensemble of lists of lists
    ensemble = [data[i*sample_size:(i+1)*sample_size] for i in range(num_groups)]
    ensemble_list = ensemble
    return ensemble_list

In [10]:
def amplitude_gen(list):
    """This function takes the real element list and the imaginary element list and returns the amplitude list"""
    amplitude_list_temp = []
    for i in range(0, len(list), 2):
        amplitude_list_temp.append(math.sqrt(list[i]**2+list[i+1]**2)/64)
    return amplitude_list_temp

In [11]:
def sum_2d_lists(list1, list2):
    if len(list1) != len(list2) or len(list1[0]) != len(list2[0]):
        return "Error: Lists must have the same dimensions"

    result = [[list1[i][j] + list2[i][j] for j in range(len(list1[0]))] for i in range(len(list1))]
    return result

In [12]:
def divide_3d_list(lst, divisor):
    return [[[element / divisor for element in sublist] for sublist in row] for row in lst]


In [13]:
temporal_multiplier = 1

In [14]:
#create list with window consideration
def createList(string, attributeJson="Micro"):
    list_activity = []
    list_rssi = []
    dict_activity = createDict(string, attributeJson)
    dict_rssi = createDict(string, attributeJson, 'rssi')
    k=0
    for i in dict_activity.keys():
        for j in (dict_activity[i].keys()):
            if(k%5 ==0):
                list_activity.append(amplitude_gen(dict_activity[i][j]))
                list_rssi.append(dict_rssi[i][j])
            k+=1
    diff_activity = fix_array(rearrange_data(np.diff(list_activity, axis =0), sample_size= int(56*temporal_multiplier)))
    diff_rssi = divide_3d_list(fix_array(rearrange_data(np.repeat(np.diff(list_rssi, axis =0), 56, axis = 1), sample_size= int(56*temporal_multiplier))), 30)
    return np.add(0, diff_activity)#diff_rssi

In [15]:
#pet
ahmed_pet_list_ch_1 = createList(string_ahmed_pet_ch_1)
ahmed_pet_list_ch_2 = createList(string_ahmed_pet_ch_2)
ahmed_pet_list_ch_3 = createList(string_ahmed_pet_ch_3)
ahmed_pet_list_ch_4 = createList(string_ahmed_pet_ch_4)

anica_pet_list_ch_1 = createList(string_anica_pet_ch_1)

benjamin_pet_list_ch_1 = createList(string_benjamin_pet_ch_1)
benjamin_pet_list_ch_2 = createList(string_benjamin_pet_ch_2)

cederic_pet_list_ch_1 = createList(string_cederic_pet_ch_1)
julien_pet_list_ch_1 = createList(string_julien_pet_ch_1)
julien_pet_list_ch_2 = createList(string_julien_pet_ch_2)

johanna_pet_list_ch_1 = createList(string_johanna_pet_ch_1)
ambre_pet_list_ch_1 = createList(string_ambre_pet_ch_1)

emeric_pet_list_ch_1 = createList(string_emeric_pet_ch_1)
emeric_pet_list_ch_2 = createList(string_emeric_pet_ch_2)
emeric_pet_list_ch_3 = createList(string_emeric_pet_ch_3)


In [16]:
#human
ahmed_walk_list_1 = createList(string_ahmed_walk_ch_1, "Micro")
ahmed_run_list_1 = createList(string_ahmed_run_ch_1, "Micro")
ahmed_no_activity_list_1 = createList(string_ahmed_no_activity_ch_1, "Micro")

ahmed_walk_list_2 = createList(string_ahmed_walk_ch_2, "Micro")
ahmed_run_list_2 = createList(string_ahmed_run_ch_2, "Micro")
ahmed_no_activity_list_2 = createList(string_ahmed_no_activity_ch_2, "Micro")

ahmed_walk_list_3 = createList(string_ahmed_walk_ch_3, "Micro")
ahmed_run_list_3 = createList(string_ahmed_run_ch_3, "Micro")
ahmed_no_activity_list_3 = createList(string_ahmed_no_activity_ch_3, "Micro")

siwar_walk_list_1 = createList(string_siwar_walk_ch_1, "timestampMicroseconds")
siwar_run_list_1 = createList(string_siwar_run_ch_1, "timestampMicroseconds")
siwar_no_activity_list_1 = createList(string_siwar_no_activity_ch_1, "Micro")

siwar_walk_list_2 = createList(string_siwar_walk_ch_2, "Micro")
siwar_run_list_2 = createList(string_siwar_run_ch_2, "Micro")
siwar_no_activity_list_2 = createList(string_siwar_no_activity_ch_2, "Micro")

olivier_walk_list_ch2 = createList(string_olivierB_walk_ch_2, "Micro")
olivier_run_list_ch2 = createList(string_olivierB_run_ch_2, "Micro")
olivier_no_activity_list_ch2 = createList(string_olivierB_no_activity_ch_2, "Micro")

arnaud_walk_list_ch_1 = createList(string_arnaud_walk_ch_1, "Micro")
arnaud_run_list_ch_1 = createList(string_arnaud_run_ch_1, "Micro")
arnaud_no_activity_list_ch_1 = createList(string_arnaud_no_activity_ch_1, "Micro")

arnaud_walk_list_ch_3 = createList(string_arnaud_walk_ch_3, "Micro")
arnaud_run_list_ch_3 = createList(string_arnaud_run_ch_3, "Micro")
arnaud_no_activity_list_ch_3 = createList(string_arnaud_no_activity_ch_3, "Micro")

arnaud_walk_list_ch_2 = createList(string_arnaud_walk_ch_2, "Micro")
arnaud_run_list_ch_2 = createList(string_arnaud_run_ch_2, "Micro")
arnaud_no_activity_list_ch_2 = createList(string_arnaud_no_activity_ch_2, "Micro")


In [17]:
cederic_no_activity_list_ch_1 = createList(string_cederic_no_activity_ch_1)
cederic_walk_list_ch_1 = createList(string_cederic_walk_ch_1)
cederic_run_list_ch_1 = createList(string_cederic_run_ch_1)

julien_no_activity_list_ch_1 = createList(string_julien_no_activity_ch_1)
julien_walk_list_ch_1 = createList(string_julien_walk_ch_1)
julien_run_list_ch_1 = createList(string_julien_run_ch_1)

julien_walk_list_ch_2 = createList(string_julien_walk_ch_2)
julien_run_list_ch_2 = createList(string_julien_run_ch_2)

johanna_no_activity_list_ch_1 = createList(string_johanna_no_activity_ch_1)
johanna_walk_list_ch_1 = createList(string_johanna_walk_ch_1)
johanna_run_list_ch_1 = createList(string_johanna_run_ch_1)

ambre_no_activity_list_ch_1 = createList(string_ambre_no_activity_ch_1)
ambre_walk_list_ch_1 = createList(string_ambre_walk_ch_1)
ambre_run_list_ch_1 = createList(string_ambre_run_ch_1)

emeric_no_activity_list_ch_1 = createList(string_emeric_no_activity_ch_1)
emeric_walk_list_ch_1 = createList(string_emeric_walk_ch_1)
emeric_run_list_ch_1 = createList(string_emeric_run_ch_1)

In [18]:
walk_train = []
walk_test = []
run_train = []
run_test = []
no_activity_train = []
no_activity_test = []
pet_train = []
pet_test = []

In [19]:
no_activity_total = np.concatenate([ahmed_no_activity_list_1 , ahmed_no_activity_list_2 , ahmed_no_activity_list_3 , siwar_no_activity_list_1 , siwar_no_activity_list_2 , olivier_no_activity_list_ch2 , arnaud_no_activity_list_ch_1 , arnaud_no_activity_list_ch_3 , arnaud_no_activity_list_ch_2 , cederic_no_activity_list_ch_1 , julien_no_activity_list_ch_1 , johanna_no_activity_list_ch_1 , ambre_no_activity_list_ch_1 , emeric_no_activity_list_ch_1], axis=0)
walk_total = np.concatenate([ahmed_walk_list_1 , ahmed_walk_list_2 , ahmed_walk_list_3 , siwar_walk_list_1 , siwar_walk_list_2 , olivier_walk_list_ch2 , arnaud_walk_list_ch_1 , arnaud_walk_list_ch_3 , arnaud_walk_list_ch_2 , cederic_walk_list_ch_1 , julien_walk_list_ch_1 , johanna_walk_list_ch_1 , ambre_walk_list_ch_1 , emeric_walk_list_ch_1 , julien_walk_list_ch_2], axis=0)
run_total = np.concatenate([ahmed_run_list_1 , ahmed_run_list_2 , ahmed_run_list_3 , siwar_run_list_1 , siwar_run_list_2 , olivier_run_list_ch2 , arnaud_run_list_ch_1 , arnaud_run_list_ch_3 , arnaud_run_list_ch_2 , cederic_run_list_ch_1 , julien_run_list_ch_1 , johanna_run_list_ch_1 , ambre_run_list_ch_1 , emeric_run_list_ch_1 , julien_run_list_ch_2], axis=0)
pet_total = np.concatenate([ahmed_pet_list_ch_1, ahmed_pet_list_ch_2 , ahmed_pet_list_ch_3 , ahmed_pet_list_ch_4 , anica_pet_list_ch_1 , benjamin_pet_list_ch_1, benjamin_pet_list_ch_2, cederic_pet_list_ch_1, julien_pet_list_ch_1, julien_pet_list_ch_2, johanna_pet_list_ch_1, ambre_pet_list_ch_1, emeric_pet_list_ch_1, emeric_pet_list_ch_2 , emeric_pet_list_ch_3], axis=0)

In [20]:
len(cederic_no_activity_list_ch_1)

46

In [21]:
label_walk_total = [1]*len(walk_total)
#label_run_total = [1]*len(run_total)
label_pet_total = [2]*len(pet_total)
label_no_activity_total = [0]*len(no_activity_total)

In [22]:
activity_total = []
label_total = []
activity_total.extend(no_activity_total)
label_total.extend(label_no_activity_total)
activity_total.extend(walk_total)
label_total.extend(label_walk_total)
activity_total.extend(pet_total)
label_total.extend(label_pet_total)

In [23]:
percentageTrain = 0.6
percentageTest = 0.4
x_train = np.concatenate([no_activity_total[:int(percentageTrain*len(no_activity_total))] , walk_total[:int(percentageTrain*len(walk_total))] , pet_total[:int(percentageTrain*len(pet_total))]], axis=0)
y_train = np.concatenate([label_no_activity_total[:int(percentageTrain*len(label_no_activity_total))] , label_walk_total[:int(percentageTrain*len(label_walk_total))] , label_pet_total[:int(percentageTrain*len(label_pet_total))]], axis=0)

x_test = np.concatenate([no_activity_total[int(percentageTrain*len(no_activity_total)):], walk_total[int(percentageTrain*len(walk_total)):], pet_total[int(percentageTrain*len(pet_total)):]], axis=0)
y_test = np.concatenate([label_no_activity_total[int(percentageTrain*len(label_no_activity_total)):], label_walk_total[int(percentageTrain*len(label_walk_total)):], label_pet_total[int(percentageTrain*len(label_pet_total)):]], axis=0)

In [24]:
len(walk_total)

882

In [25]:
x_train_array = np.expand_dims(np.array(x_train), axis=-1)
y_train_array = np.array(y_train)
x_test_array = np.expand_dims(np.array(x_test), axis=-1)
y_test_array = np.array(y_test)

In [ ]:
from tensorflow.keras.utils import to_categorical
DROPOUT = 0.0
LEARING_RATE = 1e-4
SAMPLING = 200
# Define the input for 2D data
input_layer = Input(shape=(temporal_multiplier*56, 56, 1))
for i in range(SAMPLING):
    noise_layer = tf.keras.layers.GaussianNoise(stddev=1,seed=i)(input_layer)*(1/SAMPLING) + input_layer
    input_layer = noise_layer
#noise_layer = tf.keras.layers.GaussianNoise(stddev=1)(input_layer)

# Define the model architecture
x = Conv2D(filters=4*2, kernel_size=4, activation='relu')(input_layer)
# x = BatchNormalization()(x)
x = Dropout(DROPOUT)(x)
x = MaxPooling2D(pool_size=1)(x)
x = Conv2D(filters=3*1, kernel_size=4, activation='relu')(x)
x = Dropout(DROPOUT)(x)
# x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=4*2, kernel_size=4, activation='relu')(x)
x = Dropout(DROPOUT)(x)
# x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=1)(x)
x = Flatten()(x)

output_layer = Dense(3, activation='softmax')(x)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARING_RATE),
            loss=CategoricalCrossentropy(),
            metrics=['accuracy'])

# Set your desired threshold for accuracy
desired_accuracy = 0.95

class AccuracyThresholdCallback(Callback):
    def __init__(self, threshold):
        super(AccuracyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        if logs is not None and logs.get('accuracy') is not None:
            current_accuracy = logs.get('accuracy')
            if current_accuracy >= self.threshold:
                print(f"\nReached {self.threshold * 100}% accuracy so cancelling training!")
                self.model.stop_training = True

# Initialize your custom callback
accuracy_threshold_callback = AccuracyThresholdCallback(desired_accuracy)

# Train the model with the custom callback
history=model.fit(x_train_array,to_categorical(np.array(y_train)), epochs=55*25,
        batch_size=128, callbacks=[accuracy_threshold_callback], validation_data=(x_test_array, to_categorical(np.array(y_test))))

    # # One-hot encode your labels

# Plot training & validation loss
plt.figure(figsize=(12, 5))

# Plotting Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['accuracy'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train Val', 'Train accuracy', 'Val Loss', 'Val accuracy'], loc='upper left')

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 56, 56, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 53, 53, 8)         136       
                                                                 
 dropout (Dropout)           (None, 53, 53, 8)         0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 53, 53, 8)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 3)         387       
                                                                 
 dropout_1 (Dropout)         (None, 50, 50, 3)         0         
                                                             

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

predictions = model.predict(x_test)
predicted_classes = np.argmax(predictions, axis=1)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print(f'Accuracy: {accuracy * 100:.2f}%')